In [25]:
import pandas as pd
import re
import operator
import calendar
# Get Tourneys
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:
        row=i.split('\t')
        if(len(row[0])>7 and len(row)==1 and ('COACH' not in i and 'OVERALL' not in i and 'ECAC:' not in i and 'CAPTAIN' not in i and 'HOCKEY' not in i and 'NEIHL:' not in i and 'forfeit' not in i)):
            tourneys.append(row[0])
            
            
tourneyDict={}
for i in tourneys:
    if(i=='Key to Tournaments'):
        continue
    tourney=i.split(' ')
    tourneyDict[tourney[0]]=' '.join(tourney[1:])
tourneyDict['nc']='Non-Colleigate'
tourneyDict['Oly nc']='1932 NEAAU Olympic tryouts-Non-Colleigate'
tourneyDict['ex']='Exhibition'
tourneyDict['HF ex']='Hall of Fame Game-Exhibition'
tourneyDict['IB ex']='Ice Breaker-Exhibition'

In [26]:
# Get Games
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
gameList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:      
        if(len(i)==7):
            season=i
        if(len(row[0])>7 and len(row)==1 and ('COACH' not in i and 'OVERALL' not in i and 'ECAC:' not in i and 'CAPTAIN' not in i and 'HOCKEY' not in i and 'NEIHL:' not in i and 'forfeit' not in i)):
            tourneys.append(row[0])
            
        coachSearch=re.search("COACH: (.*)",i)
        if coachSearch != None:
            coach=coachSearch.group(1)
        captSearch=re.search("CAPTAIN.?: (.*)",i)
        if captSearch != None:
            capt=captSearch.group(1) 
            #print(capt)
        confSearch=re.search("(NEIHL|ECAC|HOCKEY EAST):",i)
        if confSearch != None:
            conf=confSearch.group(1)
            if(conf=='HOCKEY EAST'):
                conf='Hockey East'
        if(re.search('\*',i)!=None):
            gameType='Conference'
        else:
            gameType='Non-Conference'
        i=i.replace("* ",'')
        note=''
        if('†' in i):
            note='Loss by forfeit (ineligible player)'
            i=i.replace('†','')
        if('+' in i):
            note='Win by forfeit (opponent left ice – score was 5-1, BU)'
            i=i.replace('+','')
        if('‡' in i):
            note='Win by forfeit (ineligible player)'
            i=i.replace('‡','')
        game=re.search(r"(\d*\/\d*) (\w*) (?:\((.?ot)\))? ?(.*)\t(\S*|\S* \S*|\S* \S* \S*) ?(\(.*\))? (\d*-\d*)",i)
        if game==None:
            continue
        #print(game.groups())
        gameDict={'date':game.group(1),
                 'result':game.group(2),
                 'ot':game.group(3),
                 'arena':game.group(4),
                 'opponent':game.group(5),
                 'gameType':gameType,
                 'tourney': game.group(6),
                 'scoreline':game.group(7),
                 'location':'',
                 'coach':coach,
                 'captain':capt,
                 'conference':conf,
                 'season':season,
                 'note':note}
        if(gameDict['gameType']==None):
            gameDict['gameType']='Non-Conference'
        if(gameDict['tourney']=='(ex)' or gameDict['result']=='E'):
            gameDict['gameType']='Exhibition'
            gameDict['result'] = 'E'
            gameDict['tourney'] = None
        if(gameDict['arena']=='Agganis Arena' or gameDict['arena']=='Brown Arena' or gameDict['arena']=='Boston Arena'):
            gameDict['location']='Home'
        elif(gameDict['tourney']==None or gameDict['tourney']=='(nc)' or gameDict['tourney'] == '(B1G/HE)' or ((gameDict['tourney'] == '(HE)' or gameDict['tourney'] == '(ECAC)') and (gameDict['arena'] != 'TD Garden' and gameDict['arena'] != 'Boston Garden' and gameDict['arena'] != 'Providence CC'))):
            gameDict['location']='Away'
        if(gameDict['location']=='' or gameDict['arena']=='Boston Garden' or gameDict['arena']=='VW Arena'):
            gameDict['location']='Neutral'
        if((gameDict['arena']=='Gutterson' and gameDict['opponent']=='Vermont') or (gameDict['arena']=='Houston' and gameDict['opponent']=='Rensselaer') or (gameDict['arena']=='Broadmoor' and gameDict['opponent']=='Colorado College') or (gameDict['arena']=='DEC Center' and gameDict['opponent']=='Minnesota Duluth')or (gameDict['arena']=='Magness Arena' and gameDict['opponent']=='Denver')or (gameDict['arena']=='Mariucci Arena' and gameDict['opponent']=='Minnesota')or (gameDict['arena']=='Munn Ice Arena' and gameDict['opponent']=='Michigan State')or (gameDict['arena']=='Walker Arena' and gameDict['opponent']=='Clarkson')or (gameDict['arena']=='Thompson Arena' and gameDict['opponent']=='Dartmouth')or (gameDict['arena']=='St. Louis Arena' and gameDict['opponent']=='St. Louis') or (gameDict['arena']=='Sullivan Arena' and gameDict['opponent']=='Alaska Anchorage')):
            gameDict['location']='Away'

        if(gameDict['tourney']!=None):
            #print(gameDict)
            gameDict['tourney']=tourneyDict[gameDict['tourney'].replace('(','').replace(')','')]
        gameDict['month'],gameDict['day']=gameDict['date'].split('/')
        gameDict['month']=int(gameDict['month'])
        gameDict['day']=int(gameDict['day'])
        if(gameDict['month'] >=9):
            gameDict['date']+="/" + gameDict['season'][:4]
            gameDict['year']=int(gameDict['season'][:4])
        elif(gameDict['month'] <= 4):
            gameDict['date']+= "/" + str(int(gameDict['season'][:4])+1)
            gameDict['year']=int(gameDict['season'][:4])+1
        gameDict['BUScore'],gameDict['OppoScore']=int(gameDict['scoreline'].split('-')[0]),int(gameDict['scoreline'].split('-')[1])
        gameDict['GD']=abs(gameDict['BUScore']-gameDict['OppoScore'])
        if(gameDict['season']=='1973-74' and gameDict['date']=='12/12/1973'):
            coach='Jack Parker'
        gameDict['date']=pd.Timestamp(gameDict['date'])
        gameList.append(gameDict)
f.close()
dfGames=pd.DataFrame(gameList)

In [639]:
for i in dfGames.loc[dfGames['date'].between('9/1/2012','5/1/2016')].groupby('result').size()[['W','L','T']]:
    print(i,end='-')
print('\b')

80-58-16-


In [57]:
for i in dfGames.loc[dfGames['date']>'1900'].groupby('arena').count().sort_values('date',ascending=False)['date'].keys():
    print(i)

Walter Brown Arena
Boston Arena
Agganis Arena
Boston Garden
TD Garden
Schneider Arena
Matthews Arena
Alfond Arena
Lawler Rink
Conte Forum
Gutterson Fieldhouse
Tsongas Center
RI Auditorium
Mullins Center
Baker Rink
Whittemore
Snively Arena
Houston Field House
McHugh Forum
Davis Rink
Appleton Arena
Smith Rink
Walker Arena
N. Haven Arena
Tully Forum
Ingalls Rink
DEC Center
Meehan Auditorium
Bright Center
Thompson Arena
War Mem. Aud.
Starr Rink
DU Arena
DCU Center
UNH Rink
Lynah Rink
Watson Rink
XL Center
Olympia
Broadmoor
MSG IV
Bradley Center
Providence CC
Mariucci Arena
Alfond (Colby)
St. Louis Arena
Munn Ice Arena
Engelstad Arena
Dane Cty. Col.
Pepsi Arena
Yost Ice Arena
Skating Club
Magness Arena
MSG III
Compton Arena
Beaverbrook
Four Seasons
N. Shore Ctr.
VW Arena
St. Paul CC
Williams Arena
Bowdoin Rink
Walter Brown Arena 
South End Arena
Kohl Center
Sullivan Arena
Fenway Park
Stud. Ice Arena
Joe Louis
Lakeview Arena
Stuart Rink
Verizon Center (Mankato)
Xcel Energy Center
Verizon Cen

In [22]:
for i in dfGames.loc[dfGames['date'].between('5/1/2016','5/1/2022')].groupby('result').size()[['W','L','T']]:
    print(i,end='-')
print('\b')

104-75-23-


In [3]:
def convertToInt(val):
        if(val.isdigit()):
            val=int(val)
        else:
            val=None
        return val

In [4]:
def convertToFloat(val):
        try:
            val=float(val)
        except:
            val=float('nan')
        return val

In [5]:
def convertSeasons(season):
        gap=season.split(',')
        years=season[2:].split('-')
        seasonStr=''
        if(len(gap)>1):
            for i in gap:
                seasonStr+=convertSeasons(i)
        else:
            yearDiff=abs(int(years[0])-int(years[1]))
            if(yearDiff>6):
                yearDiff=100-yearDiff
            firstHalf=season[:4]
            seasonStr=''
            for i in range(yearDiff):
                secondHalf=int(firstHalf)+1
                seasonStr+=str(firstHalf)+'-'+str(secondHalf)[2:]+','
                firstHalf=secondHalf
        return seasonStr[:-1]

In [6]:
def decodeTeam(team):
    origTeam = team
    team=team.lower()
    team=team.replace(" ","")
    team=team.replace("-","")
    team=team.replace("'","")
    team=team.replace(".","")
    if(team=='beanpot'):
        team = random.choice(['bu','bc','nu','hu'])
    dict={"afa" : "Air Force",
        "aic" : "American International",
        "alabamahuntsville" : "Alabama Huntsville",
        "americanintl" : "American International",
        "au" : "Augustana",
        "amworst" : "Massachusetts",
        "amwurst" : "Massachusetts",
        "anosu" : "Ohio State",
        "army" : "Army West Point",
        "asu" : "Arizona State",
        "bama" : "Alabama Huntsville",
        "bc" : "Boston College",
        "bemidji" : "Bemidji State",
        "bgsu" : "Bowling Green",
        "bigred" : "Cornell",
        "bobbymo" : "Robert Morris",
        "boston" : "Boston University",
        "bostonu" : "Boston University",
        "bowlinggreenstate" : "Bowling Green",
        "bruno" : "Brown",
        "bu" : "Boston University",
        "cambridgewarcriminalfactory" : "Harvard",
        "cc" : "Colorado College",
        "cgate" : "Colgate",
        "gate" : "Colgate",
        "chestnuthillcommunitycollege" : "Boston College",
        "chestnuthilluniversity" : "Boston College",
        "clarky" : "Clarkson",
        "cct" : "Clarkson",
        "connecticut" : "UConn",
        "cor" : "Cornell",
        "cuse" : "Syracuse",
        "darty" : "Dartmouth",
        "du" : "Denver",
        "duluth" : "Minnesota Duluth",
        "dutchpeople" : "Union",
        "ferris" : "Ferris State",
        "ferriswheel" : "Ferris State",
        "finghawks" : "North Dakota",
        "goofers" : "Minnesota",
        "hc" : "Holy Cross",
        "hu" : "Harvard",
        "howlinhuskies" : "Northeastern",
        "huntsville" : "Alabama Huntsville",
        "icebus" : "UConn",
        "keggy" : "Dartmouth",
        "lakestate" : "Lake Superior State",
        "lakesuperior" : "Lake Superior State",
        "lowell" : "UMass Lowell",
        "lowelltech" : "UMass Lowell",
        "ulowell" : "Umass Lowell",
        "lssu" : "Lake Superior State",
        "lu" : "Lindenwood",
        "liu" : "Long Island University",
        "mack" : "Merrimack",
        "mankato" : "Minnesota State",
        "mc" : "Merrimack",
        "mich" : "Michigan",
        "meatchicken" : "Michigan",
        "mnsu" : "Minnesota State",
        "mrbee" : "American International",
        "msu" : "Michigan State",
        "mtu" : "Michigan Tech",
        "nd" : "Notre Dame",
        "nebraskaomaha" : "Omaha",
        "neu" : "Northeastern",
        "newtonsundayschool" : "Boston College",
        "newhavenwarcriminalfactory" : "Yale",
        "nmu" : "Northern Michigan",
        "northern" : "Northern Michigan",
        "nodak" : "North Dakota",
        "nu" : "Northeastern",
        "osu" : "Ohio State",
        "pc" : "Providence",
        "pianohuskies" : "Michigan Tech",
        "prinny" : "Princeton",
        "psu" : "Penn State",
        "purplecows" : "Minnesota State",
        "qu" : "Quinnipiac",
        "quinny" : "Quinnipiac",
        "quinni" : "Quinnipiac",
        "rmu" : "Robert Morris",
        "rpi" : "Rensselaer",
        "rit" : "RIT",
        "saintas" : "Saint Anselm",
        "scsu" : "St. Cloud State",
        "shu" : "Sacred Heart",
        "slu" : "St. Lawrence",
        "slushbus" : "UConn",
        "smc" : "Saint Michael's",
        "sparky" : "Arizona State",
        "sparty" : "Michigan State",
        "stanselm" : "Saint Anselm",
        "stcloud" : "St. Cloud State",
        "stmichaels" : "Saint Michael's",
        "stmikes" : "Saint Michael's",
        "stthomas" : "St. Thomas",
        "ust" : "St. Thomas",
        "sootech" : "Lake Superior State",
        "su" : "Syracuse",
        "syracuse" : "Syracuse",
        "toothpaste" : "Colgate",
        "uaa" : "Alaska Anchorage",
        "uaf" : "Air Force",
        "uaf" : "Alaska",
        "uah" : "Alabama Huntsville",
        "uconn" : "UConn",
        "umass" : "Massachusetts",
        "uma" : "Massachusetts",
        "umassamherst" : "Massachusetts",
        "umasslowell" : "UMass Lowell",
        "umd" : "Minnesota Duluth",
        "uml" : "UMass Lowell",
        "umo" : "Maine",
        "umaine" : "Maine",
        "umtc" : "Minnesota",
        "umn" : "Minnesota",
        "und" : "North Dakota",
        "unh" : "New Hampshire",
        "uno" : "Omaha",
        "usma" : "Army West Point",
        "uvm" : "Vermont",
        "uw" : "Wisconsin",
        "wisco" : "Wisconsin",
        "wmu" : "Western Michigan",
        "ziggy" : "Bowling Green",
        "zoomass" : "Massachusetts"}

    if team in dict:
        return dict[team]
    else:
        teamName=''
        teamSplit = origTeam.split(' ')
        for i in range(len(teamSplit)):
            teamName+=teamSplit[i].capitalize()
            if(i<len(teamSplit)-1):
                teamName+=' '
        return teamName


In [7]:
# Get Jerseys
fileName=('JerseyNumbers.txt')
tourneys=[]
teamSet=set()
playerList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    numDict={'number':0,'player':'','season':''}
    for i in rows:
        
        numSearch=re.search("#(.*)",i)
        if numSearch != None:
            number=numSearch.group(1)
        if("Retired - " in i):
                continue
        playerSearch=re.search("(\d*-\d*) (.*)",i)
        if playerSearch != None:
            season=playerSearch.group(1)
            numDict={'number':int(number),
                 'season':convertSeasons(season),
                 'name':playerSearch.group(2)}
            playerList.append(numDict)
f.close()
dfJersey=pd.DataFrame(playerList)

In [8]:
fileName=('SkaterStats.txt')
tourneys=[]
teamSet=set()
skateList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        skaterSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)\/(\S*)',i)
        if skaterSearch!=None:
            skaterDict={'last':skaterSearch.group(1),
                       'first':skaterSearch.group(2),
                        'name':skaterSearch.group(2)+' '+skaterSearch.group(1),
                       'seasons':convertSeasons(skaterSearch.group(3)),
                       'gp':convertToInt(skaterSearch.group(4)),
                       'goals':convertToInt(skaterSearch.group(5)),
                       'assists':convertToInt(skaterSearch.group(6)),
                       'pts':convertToInt(skaterSearch.group(7)),
                       'pen':convertToInt(skaterSearch.group(8)),
                       'pim':convertToInt(skaterSearch.group(9))}
            skateList.append(skaterDict)
dfSkate=pd.DataFrame(skateList)

In [9]:
fileName=('GoalieStats.txt')
tourneys=[]
teamSet=set()
goalieList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        goalieSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)',i)
        if goalieSearch!=None:
            mins=goalieSearch.group(5).split(':')
            if(len(mins)>1):
                time = "{}:{}".format(*divmod(int(mins[0]), 60)) + ":" + mins[1]
                time = pd.to_timedelta(time)
            else:
                time=float('nan')
            goalieDict={'last':goalieSearch.group(1),
                       'first':goalieSearch.group(2),
                       'name':goalieSearch.group(2)+' '+goalieSearch.group(1),
                       'seasons':convertSeasons(goalieSearch.group(3)),
                       'gp':convertToInt(goalieSearch.group(4)),
                       'mins':round(pd.Timedelta(time).total_seconds()/60,2),
                       'ga':convertToInt(goalieSearch.group(6)),
                       'gaa':convertToFloat(goalieSearch.group(7)),
                       'saves':convertToInt(goalieSearch.group(8)),
                       'sv%':convertToFloat(goalieSearch.group(9)),
                       'W':convertToInt(goalieSearch.group(10)),
                       'L':convertToInt(goalieSearch.group(11)),
                       'T':convertToInt(goalieSearch.group(12))}
            goalieList.append(goalieDict)
dfGoalie=pd.DataFrame(goalieList)            

In [419]:
query='wins at NU under Quinn in last 10'

In [10]:
def determineQueryType(query):
    qType=''
    if('record' in query):
        qType='record'
    elif('wins' in query):
        qType='wins'
    elif('loses' in query):
        qType='loses'
    elif('ties' in query):
        qType='ties'
    return qType

In [342]:
def tokenizeQuery(query):
    keywords=['vs','at','under','since','after','between','with','before','from','in','on','against']
    keyDict={}
    for i in keywords:
        if(query.find(i+' ')>=0):
            finds=[m.start() for m in re.finditer(i+' ', query)]
            if(finds==[]):
                continue
            elif(len(finds)>1):
                counter=0
                for d in range(len(finds)):
                    if(finds[d]==0):
                        finds[d]=1
                        query=' '+query
                    keyDict[i+'_'+str(counter)]=finds[d]
                    counter+=1
            else:
                if(finds[0]==0):
                    finds[0]=1
                    query=' '+query
                keyDict[i]=finds[0]
    sorted_key = sorted(keyDict.items(), key=operator.itemgetter(1))
    keyWordList=[]
    for i in sorted_key:
        keyWordList.append(i[0])
        keyWordsDict={}
    tokens=re.split(" | ".join(keywords),query)
    if(tokens[0]==' ' or tokens[0]==''):
        tokens.pop(0)
    for i in range(len(tokens)):
        tokens[i]=tokens[i].rstrip(' ').lstrip(' ')
        keyWordsDict[keyWordList[i]]=tokens[i]
    return keyWordsDict

In [343]:
def cleanupQuery(query,qType):
    cleanlist=['the','of',"bu","bu's",'what','is',"what's",'number of','games','game','his','arena','rink']
    for i in cleanlist:
        query=query.replace(i+' ','')
    query=query.replace(qType+' ','')
    return query

In [354]:
def getResults(dfGames,query):
    query=query.lower()
    months=[x.lower() for x in list(calendar.month_name)]
    months_short=[x.lower() for x in list(calendar.month_abbr)]
    qType=determineQueryType(query)
    queryDict=tokenizeQuery(cleanupQuery(query,qType))
    print(queryDict)
    dfQueryList = []
    if('vs' in queryDict.keys()):
        print("Opponent is:",decodeTeam(queryDict['vs']))
        dfQueryList.append("(dfGames['opponent'].str.contains('{}',case=False))".format(decodeTeam(queryDict['vs'])))
    if('against' in queryDict.keys()):
        print("Opponent is:",decodeTeam(queryDict['against']))
        dfQueryList.append("(dfGames['opponent'].str.contains('{}',case=False))".format(decodeTeam(queryDict['against'])))
    if('under' in queryDict.keys()):
        print('Coach is: ', queryDict['under'])
        if(queryDict['under']=='AOC'.lower()):
            queryDict['under']="Albie O'Connell"
        dfQueryList.append("(dfGames['coach'].str.contains(\"{}\",case=False))".format(queryDict['under']))
    if('with' in queryDict.keys()):
        print('Coach is: ', queryDict['with'])
        if(queryDict['with']=='AOC'.lower()):
            queryDict['with']="Albie O'Connell"
        dfQueryList.append("(dfGames['coach'].str.contains(\"{}\",case=False))".format(queryDict['with']))
    if('at' in queryDict.keys()):
        if(queryDict['at'].capitalize() in ['Home','Away','Neutral']):
            print('Location is:',queryDict['at'])
            dfQueryList.append("(dfGames['location']==(\"{}\"))".format(queryDict['at']))
        elif(dfGames.loc[(dfGames['opponent']==decodeTeam(queryDict['at']))]['opponent'].count()>0):
            print('Opponent is:',decodeTeam(queryDict['at']))
            dfQueryList.append("(dfGames['opponent'].str.contains('{}',case=False))".format(decodeTeam(queryDict['at'])))
            print('Location is:','Away')
            dfQueryList.append("(dfGames['location']=='Away')".format(queryDict['at']))
        else:
            print('Location is: ',queryDict['at'])
            dfQueryList.append("(dfGames['arena'].str.contains(\"{}\",case=False))".format(queryDict['at']))
    if('in' in queryDict.keys()):
        digSearch=re.search('\d',queryDict['in'])
        decSearch=re.search('(\d{2,4})s',queryDict['in'])
        if(digSearch==None):
            print('Tourney is:',queryDict['in'])
            if('NCAA' in queryDict['in'].upper()):
                queryDict['in']=queryDict['in'].replace('s','')
            if(queryDict['in'].upper() in tourneyDict.keys()):
                queryDict['in']=tourneyDict[queryDict['in'].upper()]
            dfQueryList.append("(dfGames['tourney'].str.contains(\"{}\",case=False))".format(queryDict['in']))
        elif(decSearch!=None):
            if(len(decSearch.group(1))==2):
                if(int(decSearch.group(1))>20):
                    decadeStart=int("19"+decSearch.group(1))
                else:
                    decadeStart=int("20"+decSearch.group(1))
            else:
                decadeStart=int(decSearch.group(1))
            dfQueryList.append("(dfGames['date'].between('{}','{}'))".format(decadeStart,decadeStart+9))
                
        elif('first' in queryDict['in']):
            numGamesSearch=re.search('first (\d*)',queryDict['in'])
            if(numGamesSearch!=None):
                print('First ',numGamesSearch.group(1))
        elif('last' in queryDict['in']):
            numGamesSearch=re.search('last (\d*)',queryDict['in'])
            if(numGamesSearch!=None):
                print('last:',numGamesSearch.group(1))
        else:
            print('year: ',queryDict['in'])
            if(queryDict['in'].isdigit()):
                dfQueryList.append("(dfGames['year']=={})".format(queryDict['in']))
            elif(queryDict['in'] in months):                
                dfQueryList.append("(dfGames['month']==\"{}\")".format(months.index(queryDict['in'])))
            elif(queryDict['in'] in months_short):
                dfQueryList.append("(dfGames['month']=={})".format(months_short.index(queryDict['in'])))
                                                         
    if('since' in queryDict.keys()):
        print('Since: ',queryDict['since'])
        dfQueryList.append("(dfGames['date']>'{}')".format(queryDict['since']))
            
    if('before' in queryDict.keys()):
        print('before: ',queryDict['before'])
        dfQueryList.append("(dfGames['date']<'{}')".format(queryDict['before']))
    if('after' in queryDict.keys()):
        print('after: ',queryDict['after'])
        dfQueryList.append("(dfGames['date']>'{}')".format(queryDict['after']))
    if('between' in queryDict.keys()):
        print('Between: ',queryDict['between'].split(' and '))
        dates=queryDict['between'].split(' and ')
        dfQueryList.append("(dfGames['date'].between('{}','{}'))".format(dates[0],dates[1]))
    if('from' in queryDict.keys()):
        print('From: ',queryDict['from'].split(' to '))
        dates=queryDict['from'].split(' to ')
        dfQueryList.append("(dfGames['date'].between('{}','{}'))".format(dates[0],dates[1]))
    dfQuery =''
    for i in dfQueryList:
        dfQuery += i + " & "
    dfQuery=dfQuery[:-2]
    result=''
    print(dfQuery)
    if(qType=='record'):
        for i in ['W','L','T']:
            result+=str(eval("dfGames.loc[{} & (dfGames['result']=='{}')]['result']".format(dfQuery,i)).count())+'-'
    if(qType=='wins'):
        for i in ['W']:
            result+=str(eval("dfGames.loc[{} & (dfGames['result']=='{}')]['result']".format(dfQuery,i)).count())+'-'
    elif(qType=='loses'):
        for i in ['L']:
            result+=str(eval("dfGames.loc[{} & (dfGames['result']=='{}')]['result']".format(dfQuery,i)).count())+'-'
    elif(qType=='ties'):
        for i in ['T']:
            result+=str(eval("dfGames.loc[{} & (dfGames['result']=='{}')]['result']".format(dfQuery,i)).count())+'-'
    if(result!=''):   
        return result[:-1]
    else:
        return "No Results Found"

In [356]:
query='record vs BC in the NCAAs'
getResults(dfGames,query)


{'vs': 'bc', 'in': 'ncaas'}
Opponent is: Boston College
Tourney is: ncaas
(dfGames['opponent'].str.contains('Boston College',case=False)) & (dfGames['tourney'].str.contains("NCAA Tournament",case=False)) 


'1-1-0'

In [243]:
'GLI' in tourneyDict.keys()

True

In [357]:
dfGames.loc[(dfGames['opponent'].str.contains('Boston College',case=False)) & (dfGames['tourney'].str.contains("NCAA Tournament",case=False))]

,date,result,ot,arena,opponent,gameType,tourney,scoreline,location,coach,captain,conference,season,note,month,day,year,BUScore,OppoScore,GD
1104,1978-03-25,W,None,Providence CC,Boston College,Non-Conference,NCAA Tournament,5-3,Neutral,Jack Parker,"Brian Durocher, Jack O'Callahan",ECAC,1977-78,,3,25,1978,5,3,2
2177,2006-03-25,L,None,DCU Center,Boston College,Non-Conference,NCAA Tournament,0-5,Neutral,Jack Parker,"David Van der Gulik, Brad Zancanaro",Hockey East,2005-06,,3,25,2006,0,5,5


In [583]:
qWords=query.split(' ')
for i in qWords:
    team=decodeTeam('')
    if(dfGames.loc[(dfGames['opponent']==decodeTeam(i))]['opponent'].count()>0):
        print(i)

BC


In [39]:
dfSkate.loc[dfSkate['name'].isin((dfJersey.loc[dfJersey['number']==3]['name']))].sort_values('gp',ascending=False)

,last,first,name,seasons,gp,goals,assists,pts,pen,pim
641,Schaeffer,Kevin,Kevin Schaeffer,"2003-04,2004-05,2005-06,2006-07",152.0,17,37,54,39.0,86.0
540,Oksanen,Ahti,Ahti Oksanen,"2012-13,2013-14,2014-15,2015-16",150.0,51,64,115,30.0,68.0
538,Octeau,Jay,Jay Octeau,"1983-84,1984-85,1985-86,1986-87",143.0,16,70,86,73.0,149.0
647,Shattenkirk,Kevin,Kevin Shattenkirk,"2007-08,2008-09,2009-10",121.0,18,60,78,58.0,116.0
7,Amonte,Ty,Ty Amonte,"2017-18,2018-19,2019-20,2020-21,2021-22",119.0,17,20,37,32.0,91.0
134,Cotter,Bill,Bill Cotter,"1977-78,1978-79,1979-80,1980-81",117.0,30,54,84,54.0,143.0
547,O’Sullivan,Chris,Chris O’Sullivan,"1992-93,1993-94,1994-95,1995-96",113.0,40,88,128,50.0,124.0
741,Whitney,Ryan,Ryan Whitney,"2001-02,2002-03,2003-04",107.0,16,43,59,60.0,152.0
201,Dunbar,Dale,Dale Dunbar,"1980-811982-83,1983-84,1984-8",104.0,5,41,46,73.0,157.0
226,Fay,Gary,Gary Fay,"1973-74,1974-75,1975-76,1976-77",101.0,13,89,102,19.0,46.0
